<a href="https://colab.research.google.com/github/HannanTechy/portfolio-optimization/blob/main/6equities_code_for_PO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install investpy
!pip install hvplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.6 MB/s eta 0:00:00


In [8]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import numpy as np
import pandas as pd
import hvplot.pandas
#For Monte Carlo
import random
#Visualization
import holoviews as hv
from tqdm import tqdm
#Historical Data
import investpy
import yfinance as yf

In [36]:
stocks = ["TATAMOTORS", "INFY", "ASIANPAINT", "RELIANCE", "HDFCBANK", "ADANIPORTS"]


In [37]:
begin_date = "2014-01-01"
end_date = "2019-12-29"

In [38]:
prices = pd.DataFrame()

for stock in stocks:
    ticker = yf.Ticker(stock+".NS")
    data = ticker.history(start=begin_date, end=end_date)
    data["Ticker"] = stock
    prices = pd.concat([prices, data], axis=0)

prices = prices.reset_index()
prices = prices[["Date", "Ticker", "Close"]]
prices = prices.pivot(index="Date", columns="Ticker", values="Close")

print(prices.head())

Ticker                     ADANIPORTS  ASIANPAINT    HDFCBANK        INFY  \
Date                                                                        
2014-01-01 00:00:00+05:30  149.391891  468.446075  312.777771  346.774048   
2014-01-02 00:00:00+05:30  145.903229  455.135529  308.945343  348.028992   
2014-01-03 00:00:00+05:30  143.274750  458.838196  311.813751  356.488342   
2014-01-06 00:00:00+05:30  144.182785  459.916046  311.155457  351.763641   
2014-01-07 00:00:00+05:30  139.499313  459.259949  312.425079  345.689178   

Ticker                       RELIANCE  TATAMOTORS  
Date                                               
2014-01-01 00:00:00+05:30  414.639221  369.242859  
2014-01-02 00:00:00+05:30  408.201233  366.682404  
2014-01-03 00:00:00+05:30  403.186218  357.179291  
2014-01-06 00:00:00+05:30  398.847626  361.364563  
2014-01-07 00:00:00+05:30  392.852875  359.542786  


In [39]:
returns = prices.pct_change()
returns.head()

Ticker,ADANIPORTS,ASIANPAINT,HDFCBANK,INFY,RELIANCE,TATAMOTORS
Date,,,,,,
2014-01-01 00:00:00+05:30,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-02 00:00:00+05:30,-0.023352,-0.028414,-0.012253,0.003619,-0.015527,-0.006934
2014-01-03 00:00:00+05:30,-0.018015,0.008135,0.009285,0.024306,-0.012286,-0.025916
2014-01-06 00:00:00+05:30,0.006338,0.002349,-0.002111,-0.013253,-0.010761,0.011718
2014-01-07 00:00:00+05:30,-0.032483,-0.001427,0.004080,-0.017269,-0.015030,-0.005041


In [40]:
cov = returns.cov()
cov.head()

Ticker,ADANIPORTS,ASIANPAINT,HDFCBANK,INFY,RELIANCE,TATAMOTORS
Ticker,,,,,,
ADANIPORTS,0.000512,0.000083,0.000082,0.000030,0.000107,0.000169
ASIANPAINT,0.000083,0.000239,0.000053,0.000025,0.000066,0.000093
HDFCBANK,0.000082,0.000053,0.000120,0.000016,0.000060,0.000070
INFY,0.000030,0.000025,0.000016,0.000251,0.000025,0.000040
RELIANCE,0.000107,0.000066,0.000060,0.000025,0.000255,0.000117


In [41]:
np.random.seed(10) #for replicability
weights = np.random.random(len(stocks))
weights /= np.sum(weights)
weights

array([0.26617196, 0.00716121, 0.21866313, 0.25840174, 0.17202779,
       0.07757418])

In [42]:
rp = (returns.mean()*252)@weights 
rp

0.19243573677286005

In [43]:
port_var = weights@(cov*252)@weights 
port_var

0.029508740916038303

In [45]:
#Sharpe Ratio
rf = 0.02 #risk-free rate
sharpe = (rp-rf)/np.sqrt(port_var)
sharpe

1.0038109696227449

In [46]:
def portfolio_metrics(weights, index='Trial'):
    
    '''
    This function generates the relative performance metrics that will be reported and will be used
    to find the optimal weights.
    
    Parameters:
    weights: initialized weights or optimal weights for performance reporting
    
    '''   
    
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=[index])
    return df

In [47]:
np.random.seed(42)
#Empty Container
portfolios = pd.DataFrame(columns=[*stocks, "Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"])
#Loop
for i in range(10000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    portfolios.loc[i, stocks] = weights
    metrics = portfolio_metrics(weights,i)
    portfolios.loc[i, ["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]] = \
    metrics.loc[i,["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]]
    
portfolios

,TATAMOTORS,INFY,ASIANPAINT,RELIANCE,HDFCBANK,ADANIPORTS,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
0,0.126196,0.32033,0.246635,0.20171,0.052568,0.05256,0.213158,0.0236,0.153624,1.257345
1,0.018016,0.26867,0.186453,0.219629,0.006385,0.300846,0.140154,0.033122,0.181996,0.660203
2,0.371791,0.094836,0.081208,0.081913,0.135882,0.23437,0.159314,0.047298,0.217481,0.64058
3,0.202503,0.136533,0.286847,0.065397,0.136962,0.171757,0.187107,0.03088,0.175726,0.950949
4,0.175816,0.302687,0.076975,0.198238,0.228377,0.017907,0.22201,0.026861,0.163894,1.232565
...,...,...,...,...,...,...,...,...,...,...
9995,0.140157,0.387544,0.017094,0.160215,0.050361,0.24463,0.159022,0.036532,0.191134,0.727354
9996,0.133556,0.110197,0.301168,0.074923,0.242812,0.137343,0.199278,0.027448,0.165675,1.082107
9997,0.101867,0.061905,0.192715,0.101522,0.247333,0.294658,0.149884,0.036447,0.19091,0.680341
9998,0.094251,0.053393,0.130928,0.193833,0.124889,0.402706,0.107782,0.044131,0.210074,0.417861


In [48]:
portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()]

,TATAMOTORS,INFY,ASIANPAINT,RELIANCE,HDFCBANK,ADANIPORTS,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
6300,0.060758,0.132782,0.45011,0.160089,0.193307,0.002954,0.234253,0.020407,0.142854,1.499804


end